In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import sqlite3


## Store Wildfire occurrences data in SQL database

In [2]:
conn = sqlite3.connect('incendios_PT.db')
c = conn.cursor()

In [3]:
df_1 = pd.read_excel("data\Registos_Incendios_SGIF_2001_2010.xlsx")
df_1.head()

,Codigo_SGIF,Codigo_ANEPC,Ano,Mes,Dia,Hora,AreaPov_ha,AreaMato_ha,AreaAgric_ha,AreaTotal_ha,...,ISI,DC,DMC,FFMC,BUI,CodCausa,TipoCausa,GrupoCausa,DescricaoCausa,FonteAlerta
0,AT901106,10.0,2001,1,5,2,0.0,0.0001,0.0,0.0001,...,0.062124,6.379267,1.828987,39.942846,2.130730,NaN,NaN,NaN,NaN,CCO
1,BL201479,13557.0,2001,1,5,14,0.0,0.0100,0.0,0.0100,...,0.000036,2.404544,0.103108,13.709375,0.186251,NaN,NaN,NaN,NaN,CCO
2,TM90152,5294.0,2001,1,5,21,0.0,0.0100,0.0,0.0100,...,0.000208,2.080612,0.154052,18.319554,0.259980,NaN,NaN,NaN,NaN,CCO
3,BL101313,17112.0,2001,1,6,13,0.0,0.1000,0.0,0.1000,...,0.001751,1.462027,0.263342,25.161113,0.363155,NaN,NaN,NaN,NaN,CCO
4,BL201156,20473.0,2001,1,7,0,0.0,0.0000,0.0,0.0000,...,0.001005,1.439977,0.259641,23.521291,0.357935,NaN,NaN,NaN,NaN,CCO


In [5]:
df_1.to_sql('incendios', conn, if_exists='append', index=False)

In [6]:
df_2 = pd.read_excel("data\Registos_Incendios_SGIF_2011_2020.xlsx")
df_2.head()

,Codigo_SGIF,Codigo_ANEPC,Ano,Mes,Dia,Hora,AreaPov_ha,AreaMato_ha,AreaAgric_ha,AreaTotal_ha,...,ISI,DC,DMC,FFMC,BUI,CodCausa,TipoCausa,GrupoCausa,DescricaoCausa,FonteAlerta
0,DM2111,368,2011,1,1,17,0.0,0.010,0.00000,0.01000,...,0.291930,4.092373,0.653638,54.525424,0.934234,145.0,Negligente,Uso do fogo - Fogueiras,Fogueiras - Outras,Populares
1,BL4112,1236,2011,1,9,22,0.0,0.000,0.00200,0.00200,...,0.013831,1.835452,0.367371,32.001278,0.489703,610.0,Desconhecida,Indeterminadas,Indeterminadas - Prova material,Populares
2,DM3111,820,2011,1,15,17,0.0,0.000,0.00001,0.00001,...,1.171990,9.658571,1.573768,74.889775,2.236498,124.0,Negligente,Queimas amontoados de sobrantes florestais ou ...,Queima de amontoados - Borralheiras,Populares
3,BL2111,1977,2011,1,18,22,0.0,0.001,0.00000,0.00100,...,0.248910,12.477199,0.762519,51.102608,1.322920,124.0,Negligente,Queimas amontoados de sobrantes florestais ou ...,Queima de amontoados - Borralheiras,Populares
4,DM2113,7930,2011,1,19,17,0.0,0.005,0.00000,0.00500,...,0.008092,3.968661,0.508191,32.290416,0.769912,NaN,NaN,NaN,NaN,Populares


In [7]:
df_2.to_sql('incendios', conn, if_exists='append', index=False)

In [13]:
c.close()

## Inspect SQL database

In [4]:
%load_ext sql
%sql sqlite:///incendios_PT.db

In [23]:
a = %sql SELECT Ano, TipoCausa, SUM(AreaTotal_ha) as TotalBurnt FROM incendios GROUP BY Ano, TipoCausa; 
df = a.DataFrame()
df["norm_Area"] = df.groupby("Ano").transform(lambda x: (x / x.sum() * 100))["TotalBurnt"]
df.groupby("Ano").sum()

 * sqlite:///incendios_PT.db
Done.


,TotalBurnt,norm_Area
Ano,,
2001,117420.438657,100.0
2002,130849.078350,100.0
2003,471750.049431,100.0
2004,151369.876610,100.0
2005,346717.819232,100.0
2006,83705.863650,100.0
2007,36411.547360,100.0
2008,19897.219299,100.0
2009,92125.662431,100.0


## Add overall wildfire data to database

In [22]:
df_f = pd.read_excel("data\INE_incendios.xls", skiprows=5)
df_f = df_f.iloc[0:47]

df_f["Período de referência dos dados"] = df_f["Período de referência dos dados"].astype(float)
df_f["Período de referência dos dados"] = df_f["Período de referência dos dados"].shift(1).replace(np.nan, 0) + df_f["Período de referência dos dados"].replace(np.nan, 0)
df_f["Período de referência dos dados"] = df_f["Período de referência dos dados"].astype(int)

df_f = df_f.loc[df_f["Localização geográfica (NUTS - 2013) (1)"]=="Continente"]

df_f.rename(columns = { "Localização geográfica (NUTS - 2013) (1)": "localizacao",
                        "Período de referência dos dados": "ano",
                        "Superfície ardida (ha) por Localização geográfica (NUTS - 2013) e Tipo de superfície ardida; Anual (3) ": "supArdida_ha",
                        "Superfície ardida média (ha) por Localização geográfica (NUTS - 2013); Anual (3) ": "supArdidaMed_ha",
                        "Taxa de superfície florestal ardida (%) por Localização geográfica (NUTS - 2013); Anual": "taxaArdida_%",
                        "Incêndios rurais com duração superior a 24 horas (N.º) por Localização geográfica (NUTS - 2013); Anual (4) ": "incRurais24h_no",
                        "Incêndios rurais (N.º) por Localização geográfica (NUTS - 2013); Anual (6) ": "incRurais_no",
                        "Proporção da superfície florestal (%) por Localização geográfica (NUTS - 2013); Decenal (5) ": "propFloresta_%",

                        }, 
            inplace=True)

df_f = df_f[df_f.columns[~df_f.columns.str.contains("Unnamed")]]
df_f

,ano,localizacao,supArdida_ha,supArdidaMed_ha,taxaArdida_%,incRurais24h_no,propFloresta_%,incRurais_no
4,2021,Continente,28362,3.46,NaN,21,NaN,8186
6,2020,Continente,67170,6.98,NaN,41,NaN,9619
8,2019,Continente,42084,3.89,0.681,21,NaN,10832
10,2018,Continente,44577,3.63,0.67,158,NaN,12273
12,2017,Continente,539921,25.7,8.987,394,NaN,21006
14,2016,Continente,167807,10.45,2.581,288,NaN,16104
16,2015,Continente,67200,3.42,1.073,53,36.2,19643
18,2014,Continente,22820,2.43,0.332,10,NaN,9388
20,2013,Continente,160388,6.94,2.543,147,NaN,23129
22,2012,Continente,117985,4.86,1.835,59,NaN,25352


In [23]:
conn = sqlite3.connect('incendios_PT.db')
c = conn.cursor()
df_f.to_sql('incendios_stats', conn, if_exists='append', index=False)
c.close()

In [25]:
%%sql

select * from incendios_stats limit 2;

 * sqlite:///incendios_PT.db
Done.


ano,localizacao,supArdida_ha,supArdidaMed_ha,taxaArdida_%,incRurais24h_no,propFloresta_%,incRurais_no
2021,Continente,28362,3.46,None,21,None,8186
2020,Continente,67170,6.98,None,41,None,9619


In [24]:
# to_plot = df_f.columns[2:]
# print(to_plot)

# f, ax = plt.subplots(1, len(to_plot), figsize=(len(to_plot)*7, 5))


# for i, par in enumerate(to_plot):
#     ax[i].plot(df_f["ano"].astype(int), df_f[par])
#     ax[i].set_xlabel("Ano")
#     ax[i].set_ylabel(par)
    
# plt.show()

## Add meteorological data to database

In [27]:
df_m = pd.read_excel("data\INE_meteo.xls", skiprows=5)
df_m = df_m.iloc[1:91]
df_m.rename(columns = { "Localização geográfica (NUTS - 2013)": "localizacao",
                        "Período de referência dos dados": "ano",
                        "Desvio em relação à normal da temperatura máxima do ar (°C) por Localização geográfica (NUTS - 2013); Anual": "desvioMaxT_C",
                        "Desvio em relação à normal da temperatura mínima do ar (°C) por Localização geográfica (NUTS - 2013); Anual": "desvioMinT_C",
                        "Desvio em relação à normal da temperatura média do ar (°C) por Localização geográfica (NUTS - 2013); Anual": "desvioMedT_C",
                        "Temperatura média do ar (°C) por Localização geográfica (NUTS - 2013); Anual": "Tmedia_C",
                        "Temperatura máxima do ar (°C) por Localização geográfica (NUTS - 2013); Anual": "Tmax_C",
                        "Temperatura mínima do ar (°C) por Localização geográfica (NUTS - 2013); Anual": "Tmin_C",
                        "Desvio em relação à normal da precipitação total (%) por Localização geográfica (NUTS - 2013); Anual": "desvioPrec_mm",
                        "Precipitação total (mm) por Localização geográfica (NUTS - 2013); Anual": "Prec_mm",
                        }, 
            inplace=True)
df_m["ano"] = df_m["ano"].astype(int)
df_m = df_m[df_m.columns[~df_m.columns.str.contains("Unnamed")]]
df_m

,ano,localizacao,desvioMaxT_C,desvioMinT_C,desvioMedT_C,Tmedia_C,Tmax_C,Tmin_C,desvioPrec_mm,Prec_mm
1,2020,Continente,1.45,0.47,0.96,16.22,21.95,10.49,85,746.8
2,2019,Continente,1.04,-0.41,0.32,15.58,21.54,9.61,86,755.6
3,2018,Continente,0.43,-0.21,0.11,15.37,20.93,9.81,107,939.9
4,2017,Continente,2.32,-0.18,1.07,16.33,22.82,9.84,61.4,541.3
5,2016,Continente,1.12,0.19,0.65,15.91,21.62,10.21,112.4,991.6
...,...,...,...,...,...,...,...,...,...,...
86,1935,Continente,-0.73,-1.11,-0.91,14.35,19.77,8.91,NaN,NaN
87,1934,Continente,-0.78,-0.73,-0.77,14.49,19.72,9.29,NaN,NaN
88,1933,Continente,-1.2,-0.71,-0.97,14.29,19.3,9.31,NaN,NaN
89,1932,Continente,-1.94,-0.95,-1.34,13.92,18.56,9.07,NaN,NaN


In [28]:
conn = sqlite3.connect('incendios_PT.db')
c = conn.cursor()
df_m.to_sql('meteo_stats', conn, if_exists='append', index=False)
c.close()

In [33]:
%%sql 
select * from incendios limit 3;

 * sqlite:///incendios_PT.db
Done.


Codigo_SGIF,Codigo_ANEPC,Ano,Mes,Dia,Hora,AreaPov_ha,AreaMato_ha,AreaAgric_ha,AreaTotal_ha,ClasseArea,DataHoraAlerta,DataHora_PrimeiraIntervencao,DataHora_Extincao,Duracao_Horas,IncSup24horas,DTCCFR,Distrito,Concelho,Freguesia,Local,RNAP,RNMPF,X_Militar,Y_Militar,Latitude,Longitude,X_ETRS89,Y_ETRS89,DSR,FWI,ISI,DC,DMC,FFMC,BUI,CodCausa,TipoCausa,GrupoCausa,DescricaoCausa,FonteAlerta
AT901106,10.0,2001,1,5,2,0.0,0.0001,0.0,0.0001,]0 a 1 ha[,2001-01-05 02:00:00,None,2001-01-05 02:51:00,0.85,0,20701,Beja,Cuba,Cuba,CUBA,None,None,221164.0,133302.0,38.1663888888889,-7.89138888888889,21165.5567827263,-166696.064596428,2.58060711158649e-05,0.0195964240248731,0.0621242207500858,6.37926670872986,1.82898711573961,39.9428460790888,2.13073036425057,None,None,None,None,CCO
BL201479,13557.0,2001,1,5,14,0.0,0.01,0.0,0.01,]0 a 1 ha[,2001-01-05 14:18:00,None,2001-01-05 14:23:00,0.0833333333333333,0,60326,Coimbra,Coimbra,Souselas,São Martinho do Pinheiro,None,None,174200.0,369900.0,40.2972222222222,-8.43638888888889,-25798.7862361463,69899.1225617019,2.51480316177235e-11,7.87061243714669e-06,3.64257946356077e-05,2.40454382324219,0.103108499565819,13.7093753671551,0.186250611993803,None,None,None,None,CCO
TM90152,5294.0,2001,1,5,21,0.0,0.01,0.0,0.01,]0 a 1 ha[,2001-01-05 21:00:00,None,2001-01-05 22:35:00,1.58333333333333,0,40718,Bragança,Mirandela,Lamas de Orelhão,LAMAS DE ORELHÃO,None,None,269500.0,496900.0,41.4383333333333,-7.30138888888889,69496.929086648,196897.680672135,5.72467769747031e-10,4.60060308659147e-05,0.000208124597370919,2.08061193847656,0.154051710915804,18.319554187924,0.259980104014752,None,None,None,None,CCO


In [ ]:
to_plot = df_m.columns[2:]
print(to_plot)

f, ax = plt.subplots(1, len(to_plot), figsize=(len(to_plot)*7, 5))


for i, par in enumerate(to_plot):
    ax[i].plot(df_m["ano"].astype(int), df_m[par])
    ax[i].set_xlabel("Ano")
    ax[i].set_ylabel(par)
    
plt.show()

In [ ]:
df = df_m.merge(df_f, on="ano")
df

In [ ]:
xx = "Tmedia_C"
yy = "incRurais24h_no"

sns.scatterplot(data=df, x=xx, y=yy)
df_ = df[[xx, yy]].dropna().astype(float)
x = df_[xx].values
y = df_[yy].values
slope, intercept, r, p, _ = stats.linregress(x, y)
plt.plot(x, slope*x + intercept)
print(r**2)